<a href="https://colab.research.google.com/github/rmcpantoja/My-Colab-Notebooks/blob/main/notebooks/ForwardTacotron_HiFi_Gan_Fine_tuning_Espa%C3%B1ol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [ForwardTacotron](https://github.com/as-ideas/ForwardTacotron) e [HiFi-GAN](https://github.com/jik876/hifi-gan) Cuaderno de Fine Tuning.

Cuaderno creado por [rmcpantoja.](https://github.com/rmcpantoja/)

## Créditos:

* Modificación de HiFi-GAN por [SortAnon.](https://github.com/SortAnon/hifi-gan)
* Este cuaderno se basa en el [cuaderno del equipo FakeYou.](https://colab.research.google.com/github/justinjohn0306/FakeYou-Tacotron2-Notebook/blob/main/FakeYou_HiFi_GAN_Fine_Tuning.ipynb) Por `Cookie PPP` y `Justinjohn03`.
* Traducido al español por [Xx_Nessu_xX.](https://fakeyou.com/profile/Xx_Nessu_xX)

## Primeros pasos.

In [ ]:
#@title Mirar GPU.
#@markdown Necesitas un `T4 Tesla` como mínimo.
!nvidia-smi

In [ ]:
#@title Montar Google Drive.
#@markdown Esto es útil para importar los datos necesarios para el ajuste fino en HiFi-GAN y exportar el entrenamiento de estos modelos ajustados.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#@title Instalar software.
#@markdown Esta celda instala ForwardTacotron, HiFi-GAN y algunas dependencias necesarias para ejecutar las tareas.
#@markdown #### <font color=orange>¿Quieres utilizar la versión 3.2 de ForwardTacotron?

#@markdown La versión 3.2 es la última que sólo contiene soporte para un único hablante.
only_singlespeaker_version = True #@param {type:"boolean"}
#@markdown ---
%cd /content
!git clone https://github.com/as-ideas/ForwardTacotron.git
!git clone https://github.com/SortAnon/hifi-gan.git
# rollback to last single speaker version (tag v3.2)
%cd ./ForwardTacotron
if only_singlespeaker_version.
  !git checkout 632b453c6cb6d15dfe4dd168cd60c60e56731829
!rm -r .git/
# install
!apt-get install espeak-ng
!pip install numba librosa==0.9.2 pyworld phonemizer webrtcvad PyYAML soundfile scipy tensorboard matplotlib unidecode inflect
!pip install --upgrade gdown

## Datos y entrenamiento.

In [ ]:
#@title Establecer rutas de datos.
!mkdir /content/ForwardTacotron/wavs
#@markdown #### Ruta del conjunto de datos preprocesado (Data y archivo de configuración):
project = "/content/drive/MyDrive/ForwardTacotron/ejemploTTS/dataset_preprocessed.zip" #@param {type:"string"}
print("Extracción del conjunto de datos preprocesado...")
!unzip -q "$project" -d /content/ForwardTacotron
#@markdown ---
#@markdown #### Ruta del conjunto de datos de audio (En formato zip):
audio_dataset = "/content/drive/MyDrive/datasets/Ejemplo/wavs.zip" #@param {type:"string"}
print("Extracción de audio...")
!unzip -q "$audio_dataset" -d /content/ForwardTacotron/wavs
#@markdown ---
#@markdown #### Carpeta donde se encuentran los puntos de control Tacotron o ForwardTacotron:
checkpoints_dir = "/content/drive/MyDrive/ForwardTacotron/ejemploTTS/checkpoints" #@param {type:"string"}
!cp -r "$checkpoints_dir" /content/ForwardTacotron
#@markdown ---
#@markdown #### Rutas de las listas de archivos de entrenamiento y validación:
train_filelist = "/content/drive/MyDrive/Datasets/Ejemplo/list.txt" #@param {type:"string"}
val_filelist = "/content/drive/MyDrive/Datasets/Ejemplo/list.txt" #@param {type:"string"}
#@markdown ---


In [ ]:
#@title Crear conjunto de datos alineado con la verdad fundamental.
#@markdown Con este conjunto de datos, HiFi-GAN puede distinguir el sonido del modelo a alinear con los audios originales para adaptar su sonido.

#@markdown ---
%cd /content/ForwardTacotron
# create GTA features for:
#@markdown #### Seleccione el modelo en el que deseas trabajar:
GTA_model = "forward" #@param ["forward", "tacotron"]
#@markdown ---
!python train_{GTA_model}.py --force_gta

In [ ]:
#@title Entrenar HiFi-GAN
%cd /content/hifi-gan

import os
import shutil
#@markdown #### Nombre deseado para el modelo:
model_name = "Ejemplo" #@param {type:"string"}
#@markdown ---
#@markdown #### Directorio de salida (Se recomienda guardarlo en Drive.):
output_dir = "/content/drive/My Drive/forward_finetune"  #@param {type:"string"}
savedir = output_dir+"/"+model_name
#@markdown ---
#@markdown #### Intervalo de pasos para guardar los modelos:
checkpoint_interval = 250 #@param {type:"integer"}
#@markdown ---
import gdown
d = 'https://drive.google.com/uc?id='
os.chdir('/content/hifi-gan')
# copy fine tune dataset:
if os.path.exists("/content/hifi-gan/ft_dataset"):
    shutil.rmtree("/content/hifi-gan/ft_dataset")
shutil.copytree("/content/ForwardTacotron/data/gta", "/content/hifi-gan/ft_dataset/wavs/")
if os.path.exists("/content/hifi-gan/wavs"):
    shutil.rmtree("/content/hifi-gan/wavs")
shutil.copytree("/content/ForwardTacotron/wavs/", "/content/hifi-gan/wavs/")
shutil.copyfile(train_filelist, "/content/hifi-gan/training.txt")
shutil.copyfile(val_filelist, "/content/hifi-gan/validation.txt")

if not os.path.exists(savedir):
    os.makedirs(savedir)
if not os.path.exists(os.path.join(savedir, "do_00000000")):
    print("Downloading universal model...")
    gdown.download(d+"1O63eHZR9t1haCdRHQcEgMfMNxiOciSru", os.path.join(savedir, "do_00000000"), quiet=False)
    gdown.download(d+"1qpgI41wNXFcH-iKq1Y42JlBC9j0je8PW", os.path.join(savedir, "g_00000000"), quiet=False)
    start_from_universal = "--warm_start True "
else:
    start_from_universal = ""


!python train.py --fine_tuning True --config config_v1b.json \{start_from_universal}\
--checkpoint_interval {checkpoint_interval} --checkpoint_path "{savedir}" \
--input_training_file "/content/hifi-gan/training.txt" \
--input_validation_file "/content/hifi-gan/validation.txt" \
--input_wavs_dir "" --input_mels_dir "ft_dataset"


